In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet

In [2]:
b = pd.read_csv('biomass.csv')

In [3]:
b = b[['Index','year','biomass']].copy()

In [4]:
b

,Index,year,biomass
0,0,2010,8.475744
1,0,2011,8.868568
2,0,2012,9.202181
3,0,2013,6.023070
4,0,2014,10.788374
...,...,...,...
19339,2417,2013,0.209801
19340,2417,2014,0.229067
19341,2417,2015,0.230417
19342,2417,2016,0.274289


In [5]:
import random
rand = random.randint(0,2417)

In [6]:
biomass = b[b['Index'] == rand]

In [7]:
biomass.rename(columns={'year':'ds','biomass':'y'},inplace=True)

biomass=biomass[['ds','y']].copy()

C:\Users\ROG\AppData\Local\Temp\ipykernel_3584\303923813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biomass.rename(columns={'year':'ds','biomass':'y'},inplace=True)


In [8]:
biomass['ds'].dtype


dtype('int64')

In [9]:
biomass['d'] = '0101'

In [10]:
biomass['ds']=biomass['ds'].apply(str)

In [11]:
biomass , biomass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 4544 to 4551
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ds      8 non-null      object 
 1   y       8 non-null      float64
 2   d       8 non-null      object 
dtypes: float64(1), object(2)
memory usage: 256.0+ bytes


(        ds           y     d
 4544  2010  162.090424  0101
 4545  2011  199.676697  0101
 4546  2012  221.403549  0101
 4547  2013  221.403549  0101
 4548  2014  221.403549  0101
 4549  2015  199.676697  0101
 4550  2016  156.568146  0101
 4551  2017  162.090424  0101,
 None)

In [12]:
biomass['ds']=biomass['ds']+biomass['d']
biomass = biomass[['ds','y']].copy()

In [13]:
biomass['ds']=biomass['ds'].apply(str)
biomass['ds']=pd.to_datetime(biomass['ds'])

In [14]:
biomass , biomass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 4544 to 4551
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      8 non-null      datetime64[ns]
 1   y       8 non-null      float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 192.0 bytes


(             ds           y
 4544 2010-01-01  162.090424
 4545 2011-01-01  199.676697
 4546 2012-01-01  221.403549
 4547 2013-01-01  221.403549
 4548 2014-01-01  221.403549
 4549 2015-01-01  199.676697
 4550 2016-01-01  156.568146
 4551 2017-01-01  162.090424,
 None)

In [15]:
model = Prophet(yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,n_changepoints=1,
    changepoint_range=0.9)

In [16]:
X = biomass[:6] 
y = biomass['ds'][6:] 

In [17]:
model.fit(X)

09:09:57 - cmdstanpy - INFO - Chain [1] start processing
09:09:58 - cmdstanpy - INFO - Chain [1] done processing


In [18]:
y = pd.DataFrame(y,columns=['ds'])

In [19]:
predicted_value = model.predict(y)

In [20]:
from sklearn.metrics import mean_absolute_error

In [21]:
mean_absolute_error(biomass['y'][6:] , predicted_value['yhat'])

75.44908227406492

In [22]:
predicted_value['yhat']

0    234.274279
1    235.282455
Name: yhat, dtype: float64

In [23]:
print('the station is =' ,rand)

the station is = 568


In [25]:
future_data = pd.DataFrame({'ds': ['20180101', '20190101']})
future_data,future_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ds      2 non-null      object
dtypes: object(1)
memory usage: 144.0+ bytes


(         ds
 0  20180101
 1  20190101,
 None)

In [27]:
future_data['ds'] = pd.to_datetime(future_data['ds'])

In [28]:
future_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      2 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 144.0 bytes


In [29]:
model.predict(future_data)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2018-01-01,162.393720,221.541396,264.221102,162.39372,162.393720,80.275835,80.275835,80.275835,80.275835,80.275835,80.275835,0.0,0.0,0.0,242.669555
1,2019-01-01,169.682671,230.802833,272.966512,169.68267,169.682671,82.376929,82.376929,82.376929,82.376929,82.376929,82.376929,0.0,0.0,0.0,252.059599


In [ ]:
final data = pd.DataFrame()
for i in range(2418):
    